In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import HeUniform, GlorotUniform
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from tensorflow.keras import losses
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Reshape, Dense, Multiply, Activation
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

base_dir = r""

train_dir = r""
val_dir = r""

img_height, img_width = 224, 224
batch_size = 32


train_datagen = ImageDataGenerator(
rescale = 1./255,
channel_shift_range=50.0,
brightness_range=[0.6, 1.4],
horizontal_flip=True,
zoom_range=[0.8, 1.2],
rotation_range=25,
shear_range=0.5,
)


val_datagen = ImageDataGenerator(
    rescale=1./255
)


train_generator = train_datagen.flow_from_directory(
    train_dir,              
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)



val_generator = val_datagen.flow_from_directory(
    val_dir,               
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


# MODEL 

## SE ATTENTION

def se_block(input_tensor , reduction =16):
    filters = int( input_tensor.shape[-1])

    se = GlobalAveragePooling2D()(input_tensor)

    se = Reshape((-1, 1, filters))(se)

    se = Dense(filters // reduction , activation="relu")(se)
    se = Dense(filters , activation ="sigmoid")(se)
    x = Multiply()([input_tensor,se])

    return x 


## RESIDUAL
def residual_attention_block(x, filters, kernel_size=(3,3), pool_size=(2,2)):
    shortcut = x

    # SHAPE UYUMSUZLUĞU İÇİN (04.07.2025  -- 14.25)
    
    if x.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=(1,1), padding='same')(shortcut)

    x = Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer=HeUniform())(x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer=HeUniform())(x)
    x = BatchNormalization()(x)

    x = se_block(x)

    x = Add()([shortcut, x])
    x = Activation('relu')(x)

    x = Dropout(0.2)(x)
    x = MaxPooling2D(pool_size=pool_size)(x)

    return x

## MODEL
def model_build_with_residual_attention(img_height, img_width,num_classes):
    inputs = Input(shape=(img_height, img_width, 3))

    x = residual_attention_block(inputs, 32)
    x = residual_attention_block(x, 64)
    x = residual_attention_block(x, 128)
    x = residual_attention_block(x, 128)

    x = GlobalAveragePooling2D()(x)

    x = Dense(32, activation="relu", kernel_initializer=HeUniform())(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(64, activation="relu", kernel_initializer=HeUniform())(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(128, activation="relu", kernel_initializer=HeUniform())(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(128, activation="relu", kernel_initializer=HeUniform())(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    

    outputs = Dense(num_classes  , activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer=Adam(learning_rate=5e-5),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model


img_height, img_width = 224, 224

model = model_build_with_residual_attention(img_height, img_width, num_classes=2)

model.summary()


early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1)

class_names = ["Brad Pitt", "Vijay Deverakonda"]
class_indices = {name: i for i, name in enumerate(class_names)}

y_train_labels = train_generator.classes  

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train_labels),
    y=y_train_labels
)

class_weight_dict = {i: weight for i, weight in enumerate(weights)}

print("Class weights:", class_weight_dict)

print("Sınıf dağılımı (train):", np.bincount(train_generator.classes))
print("Sınıf dağılımı (val):", np.bincount(val_generator.classes))
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks = [reduce_lr,early_stop,checkpoint],
    class_weight=class_weight_dict
)


# PRED FIGURE 

y_pred_probs = model.predict(val_generator)

y_pred = np.argmax(y_pred_probs, axis=1)

y_true = val_generator.classes

class_labels = list(val_generator.class_indices.keys())


print(classification_report(y_true, y_pred, target_names=class_labels))


plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.show()

